# 1. Carga de paquetes

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 2. RAG

## 2.1 Carga del archivo y extracción del texto

In [ ]:
# Archivo pdf en Internet
file_path = "https://www.argentina.gob.ar/sites/default/files/el_principe_feliz_-_oscar_wilde.pdf"  # ELEGIR PDF.

# Crear el cargador de PDF
loader = PyPDFLoader(file_path)

# Cargar el contenido del PDF
document = loader.load_and_split()

# Mostrar el número de páginas del documento
print(len(document))

In [ ]:
# Extraer textos
textos = [tupla.page_content for tupla in document]

# Unir textos
onedoc = " ".join(textos)

# Comporobar extensión
print(len(onedoc))

# Comprobar texto unido
print(f"\n{onedoc}")

## 2.2 Chunking Semántico

In [ ]:
# Modelo de embeddings de HuggingFace
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Determinar el umbral de división mediante percentil
semantic_chunker = SemanticChunker(
    embeddings_model,
    breakpoint_threshold_type="percentile"
)

In [ ]:
# Semantic Chunker
semantic_chunks = semantic_chunker.create_documents([onedoc])

In [ ]:
# Creación de vectorstore y guardado
semanctic_chunk_vectorstore = FAISS.from_documents(
    semantic_chunks,
    embedding=embeddings_model
)

## 2.3 Retrieval

In [ ]:
# Recuperación de la base de datos vectorial FAISS
semantic_chunk_retriever = semanctic_chunk_vectorstore.as_retriever(search_kwargs={"k": 4})

In [ ]:
# ELEGIR LA PREGUNTA QUE SE QUIERE HACER SOBRE EL PDF
pregunta = "¿Cómo se llama el cuento?"

In [ ]:
semantic_chunk_retriever.invoke(pregunta)

## 2.4 Augmented

In [ ]:
# Creación manual del template
rag_template = """
Usa el siguiente contexto para responder a la pregunta del usuario.
Si no conoces la respuesta, responde simplemente 'No lo sé'.
Responde en 3 frases y de manera concisa. Contesta en español.

User Query:
{question}

Context:
{context}
"""

# Creación del prompt para el RAG
rag_prompt = ChatPromptTemplate.from_template(rag_template)

## 2.5 Generation

In [ ]:
# Instanciación del modelo
llm = OllamaLLM(temperature=0.1, model="phi3:3.8b")  # ELEGIR EL MODELO A UTILIZAR.

In [ ]:
# Se crea la cadena usando Lang Chain Expression Language (LCEL)
# 'context': recuperación de la base de datos vectorial.
# 'question': pregunta del usuario sin modificaciones (RunnablePassthrough()).
# 'rag_prompt': el que se definió más arriba.
# 'llm': el modelo que se cargó.
# StrOutputParse(): formato más legible a la respuesta del modelo de lenguaje.
semantic_rag_chain = (
    {"context": semantic_chunk_retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Obtener respuesta del modelo
semantic_rag_chain.invoke(pregunta)